# Исследование лояльности пользователей

**Ссылка на проект:** https://public.tableau.com/shared/KTFZK27SZ?:display_count=n&:origin=viz_share_link

**Ссылка на презентацию:**
https://disk.yandex.ru/i/A0N3yC3AmlCFiw

**Цель исследования:** Определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score) среди клиентов из России.

**Задачи исследования:** проанализировать данные и дать ответы на следующие вопросы:\
Как распределены участники опроса по возрасту и полу? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?\
Какие группы пользователей наиболее лояльны к сервису? Какие менее?\
Какой общий NPS среди всех опрошенных?\
Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?\

В результате обработки данных получилась следующая таблица:\
user_id	Идентификатор клиента\
lt_day	Количество дней «жизни» клиента\
is_new	Поле хранит информацию о том, является ли клиент новым\
age	Возраст\
gender_segment	Пол (для удобства работы с полем преобразуйте значения в текстовый вид)\
os_name	Тип операционной системы\
cpe_type_name	Тип устройства\
country	Страна проживания\
city	Город проживания\
age_segment	Возрастной сегмент\
traffic_segment	Сегмент по объёму потребляемого трафика\
lifetime_segment	Сегмент по количеству дней «жизни»\
nps_score	Оценка клиента в NPS-опросе\
nps_group	Поле хранит информацию о том, к какой группе относится оценка клиента в опросе.

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
query = """
SELECT u.user_id, 
       u.lt_day,
       CASE
           WHEN u.lt_day <= 365 THEN 'Новый'
           ELSE 'Старый'
       END AS is_new,
       u.age,
       CASE
           WHEN u.gender_segment == 1 THEN 'Женщина'
           WHEN u.gender_segment == 0 THEN 'Мужчина'
           ELSE 'Неизвестно'
       END AS gender_segment,   
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,    
       SUBSTRING(a.title, 4) AS age_segment,
       SUBSTRING(t.title, 4) AS traffic_segment,
       SUBSTRING(l.title, 4) AS lifetime_segment,
       u.nps_score,
       CASE
           WHEN nps_score >= 9 THEN 'Сторонник'
           WHEN nps_score >= 7 THEN 'Нейтрал'
           ELSE 'Критик'
      END AS nps_group   
FROM user AS u
INNER JOIN location AS l ON u.location_id=l.location_id
INNER JOIN age_segment AS a ON u.age_gr_id=a.age_gr_id
INNER JOIN traffic_segment AS t ON u.tr_gr_id=t.tr_gr_id
INNER JOIN lifetime_segment AS l ON u.lt_gr_id=l.lt_gr_id;
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,Старый,45.0,Женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,Сторонник
1,A001WF,2344,Старый,53.0,Мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,Сторонник
2,A003Q7,467,Старый,57.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,Сторонник
3,A004TB,4190,Старый,44.0,Женщина,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,Сторонник
4,A004XT,1163,Старый,24.0,Мужчина,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,Сторонник
5,A005O0,5501,Старый,42.0,Женщина,ANDROID,SMARTPHONE,Россия,Омск,35-44,5-10,36+,6,Критик
6,A0061R,1236,Старый,45.0,Мужчина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,10-15,36+,10,Сторонник
7,A009KS,313,Новый,35.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,35-44,45-50,7-12,10,Сторонник
8,A00AES,3238,Старый,36.0,Женщина,ANDROID,SMARTPHONE,Россия,СанктПетербург,35-44,1-5,36+,10,Сторонник
9,A00F70,4479,Старый,54.0,Женщина,ANDROID,SMARTPHONE,Россия,Волгоград,45-54,15-20,36+,9,Сторонник


In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)